## Using Markov Chains to Generate Lyrics

[Markov chains](https://en.wikipedia.org/wiki/Markov_chain "Wikipedia Article on Markov Chains") can be used to generate lyrics but the end results are pretty bad.

In [1]:
def generate_model(text, order):
    """Loop through all the text until the last available full fragment + next letter."""
    model = {}

    for i in range(0, len(text) - order):
        fragment = text[i:i + order]
        next_letter = text[i + order]

        if fragment not in model:
            model[fragment] = {}

        if next_letter not in model[fragment]:
            model[fragment][next_letter] = 1
        else:
            model[fragment][next_letter] += 1

            return model


generate_model("Start walkin' me through your basement apartment", 1)

{' ': {'m': 1, 't': 1, 'w': 1, 'y': 1},
 "'": {' ': 1},
 'S': {'t': 1},
 'a': {'l': 1, 'r': 1},
 'e': {' ': 1},
 'g': {'h': 1},
 'h': {' ': 1, 'r': 1},
 'i': {'n': 1},
 'k': {'i': 1},
 'l': {'k': 1},
 'm': {'e': 1},
 'n': {"'": 1},
 'o': {'u': 2},
 'r': {'o': 1, 't': 1},
 't': {' ': 1, 'a': 1, 'h': 1},
 'u': {'g': 1},
 'w': {'a': 1},
 'y': {'o': 1}}

In [2]:
from random import choice


def get_next_character(model, fragment):
    """Figure out the the next fragment in the model."""
    letters = []

    for letter in model[fragment].keys():
        for times in range(0, model[fragment][letter]):
            letters.append(letter)

    return choice(letters)


get_next_character(
    generate_model("Start walkin' me through your basement apartment", 1), 'a')

'l'

In [3]:
def generate_text(text, order, length):
    model = generate_model(text, order)

    current_fragment = text[0:order]
    output = ''

    for i in range(0, length - order):
        new_character = get_next_character(model, current_fragment)
        output += new_character
        current_fragment = current_fragment[1:] + new_character

    print(output)


line = "Start walkin' me through your basement apartment"

line_len = len(line)

generate_text(line, 3, line_len - 10)

rt walkin' me through your basement


In [4]:
with open('./lyrics/not-moving-to-portland-john-roderick.txt') as fobj:
    lyrics = fobj.read()

In [5]:
generate_text(lyrics, 80, len(lyrics))

ough you claim got a plan to go
The way you're holding me, i don't feel you moving
But you got to follow your feelings
You should know i don't want to leave just to go
I'm not trying to be hard to reach
But i've put away my only suitcase

And i said, "hey, so you play too rough"
But i wimped out and i waited
To say, "so, you play too rough"

So you sing it the way you want to hear it sung
You grab the telephone a minute before it's rung
And you call out into space saying
Do you ever want to run?
Well, i want to run and i want you to come
Just as you first came to me
Wearing only perfume and a coat
And too, too much to drink

"hey, so you play too rough"
But i wimped out and i waited
To say, "so, you play too rough"

So you sing it the way you want to hear it sung
You grab the telephone a minute before it's rung
And you call out into space saying
Do you ever want to run?
Well, i want to run and i want you to come
Just as you first came to me
Wearing only perfume and a coat
And too, too 

## Using TensorFlow to Generate Lyrics

In [6]:
from pathlib import Path

lyrics_path = Path('./lyrics/')

In [7]:
lyrics_lines = []

for file_path in lyrics_path.glob('*.txt'):
    with open(file_path) as fobj:
        lines = fobj.readlines()

    for line in lines:
        lyrics_lines.append(line)

len(lyrics_lines)

41

In [8]:
import tensorflow as tf
from tensorflow.contrib import lookup
from tensorflow.python.platform import gfile

### Create the Vocabulary

In [9]:
max_lyric_length = 10
pad_word = 'ZYXW'

vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
    max_lyric_length)

vocab_processor.fit(lyrics_lines)

with gfile.Open('output/vocab.tsv', 'wb') as fobj:
    fobj.write('{}\n'.format(pad_word))

    for word, index in vocab_processor.vocabulary_._mapping.items():
        fobj.write('{}\n'.format(word.lower()))

n_words = len(vocab_processor.vocabulary_)

In [10]:
!tail ./output/vocab.tsv

are
dreaming
of
their
brakes
dreams
have
been
fake
i


In [11]:
table = lookup.index_table_from_file(
    vocabulary_file='./output/vocab.tsv',
    num_oov_buckets=1,
    vocab_size=None,
    default_value=-1)

numbers = table.lookup(
    tf.constant("Start walkin' me through your basement apartment".split()))

with tf.Session() as sess:
    tf.tables_initializer().run()

    print('{} --> {}'.format(lines[0], numbers.eval()))

FailedPreconditionError: HashTable has different value for same key. Key you has 15 and trying to add value 33
	 [[Node: string_to_index/hash_table/table_init = InitializeTableFromTextFileV2[delimiter="\t", key_index=-2, value_index=-1, vocab_size=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](string_to_index/hash_table, string_to_index/hash_table/table_init/asset_filepath)]]

Caused by op 'string_to_index/hash_table/table_init', defined at:
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in la

In [12]:
lyric_lines = tf.constant(lines)
words = tf.string_split(lyric_lines)
densewords = tf.sparse_tensor_to_dense(words, default_value=pad_word)
numbers = table.lookup(densewords)

In [13]:
padding = tf.constant([[0, 0], [0, max_lyric_length]])
padded = tf.pad(numbers, padding)
sliced = tf.slice(padded, [0, 0], [-1, max_lyric_length])

In [14]:
with tf.Session() as sess:
    tf.tables_initializer().run()
    
    print('titles={}{}'.format(lyric_lines.eval(), lyric_lines.shape))
    print('words={}'.format(words.eval()))
    print('dense={}{}'.format(densewords.eval(), densewords.shape))
    print('numbers={}{}'.format(numbers.eval(), numbers.shape))
    print('padding={}{}'.format(numbers.eval(), numbers.shape))
    print('padded={}{}'.format(padded.eval(), padded.shape))
    print('sliced={}{}'.format(sliced.eval(), sliced.shape))

FailedPreconditionError: HashTable has different value for same key. Key you has 15 and trying to add value 33
	 [[Node: string_to_index/hash_table/table_init = InitializeTableFromTextFileV2[delimiter="\t", key_index=-2, value_index=-1, vocab_size=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](string_to_index/hash_table, string_to_index/hash_table/table_init/asset_filepath)]]

Caused by op 'string_to_index/hash_table/table_init', defined at:
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in la

In [15]:
embedding_size = 10

embeds = tf.contrib.layers.embed_sequence(
    sliced, vocab_size=n_words, embed_dim=embedding_size)

In [16]:
window_size = embedding_size
stride = int(window_size / 2)

conv = tf.contrib.layers.conv2d(
    embeds, 1, window_size, stride=stride, padding='SAME')  # (?, 4, 1)

conv = tf.nn.relu(conv)  # (?, 4, 1)

words = tf.squeeze(conv, [2])  # (?, 4)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
